In [1]:
import numpy as np

In [2]:
import gensim.downloader as api
model = api.load('glove-twitter-100')
word_dict=set(model.index_to_key)

In [3]:
import nltk
from nltk.corpus import stopwords
sw_nltk = set(stopwords.words('english'))

In [4]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

In [5]:
import pandas as pd
train_df = pd.read_csv("F:/Kaggle/input/nlp-getting-started/train.csv")
test_df = pd.read_csv("F:/Kaggle/input/nlp-getting-started/test.csv")

In [6]:
import re
def remove_special_characters(input_string):
    # Use regular expressions to remove all special characters, but keep spaces
    cleaned_string = re.sub(r"[^a-zA-Z0-9\s]+", "", input_string)
    return cleaned_string

In [7]:
def clean(data,havetarget=True,meanofword=False):
    tmp_train_comments_tok=[tokenizer.tokenize(remove_special_characters(line.lower())) for line in data['text']]
    train_comments_tok=[]
    it=0
    for line in tmp_train_comments_tok:
        train_comments_tok.append([])
        for word in line:
            if word not in sw_nltk and word in word_dict:
                train_comments_tok[it].append(word)
        it+=1
    blank_comment_index=list()
    sentense_embedding=list()
    cnt=0
    for line in train_comments_tok:
        now=list()
        for word in line:
            now.append(model.get_vector(word))
        if len(now):
            if meanofword:
                sentense_embedding.append(now)
            else:
                sentense_embedding.append(np.array(now).mean(0))
        else:
            blank_comment_index.append(cnt)
        cnt+=1
    sentense_embedding=np.array(sentense_embedding)
    data=data.drop(blank_comment_index)
    if havetarget:
        return sentense_embedding,data['target']
    else:
        return sentense_embedding

In [8]:
train_sent_embedding,train_target=clean(train_df)
test_sent_embedding=clean(test_df,havetarget=False)

In [9]:
import torch
from torch import nn
from torch import optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader,TensorDataset

In [10]:
train_sentense_wordembedding,train_target=clean(train_df,meanofword=True)
test_sentence_wordembedding=clean(test_df,havetarget=False,meanofword=True)

C:\Users\28261\AppData\Local\Temp\ipykernel_14524\837246695.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentense_embedding=np.array(sentense_embedding)


In [11]:
def get_maxlen(listoflongshortlist):
    max_len=0
    for i in listoflongshortlist:
        max_len=max(max_len,len(i))
    return max_len
max_len=max(get_maxlen(train_sentense_wordembedding),get_maxlen(test_sentence_wordembedding))

In [12]:
def pad(listoflongshortlist):
    for i in range(len(listoflongshortlist)):
        listoflongshortlist[i]=((listoflongshortlist[i]-np.min(listoflongshortlist[i]))/np.max(listoflongshortlist[i])).tolist()
        while len(listoflongshortlist[i])!=max_len:
            # listoflongshortlist[i].append(np.zeros_like(listoflongshortlist[i][0]).tolist())
            listoflongshortlist[i].append(np.mean(listoflongshortlist[i],axis=0).tolist())
    return torch.Tensor(listoflongshortlist)

In [13]:
train_sentense_wordembedding=pad(list(train_sentense_wordembedding))
test_sentence_wordembedding=pad(list(test_sentence_wordembedding))

In [14]:
net=nn.Sequential(
    nn.Conv2d(1,4,kernel_size=(3,100)),nn.Dropout(),
    nn.MaxPool2d(kernel_size=(23-3+1,1)),
    nn.Flatten(1,3),
    nn.Linear(4,2),nn.Softmax(1)
)
optimizer=optim.Adam(net.parameters(),lr=0.00001)
loss=nn.CrossEntropyLoss()
device=torch.device('cuda')
net=net.to(device)
loss=loss.to(device)

In [15]:
for m in net.modules():
    if isinstance(m, (nn.Conv2d, nn.Linear)):
        nn.init.xavier_uniform_(m.weight)

In [16]:
train_target=torch.tensor(train_target.values).to(device)
train_sentense_wordembedding=train_sentense_wordembedding.to(device)
dataset=TensorDataset(train_sentense_wordembedding,train_target)
data_iter=DataLoader(dataset,batch_size=64,shuffle=True)

In [17]:
# num_epochs=800
# for epoch in range(num_epochs):
#     for x,y in data_iter:
#         optimizer.zero_grad()
#         l=loss(net(x.unsqueeze(1)),y)
#         l.mean().backward()
#         optimizer.step()
#     if (epoch+1)%50==1:
#         print(loss(net(train_sentense_wordembedding.unsqueeze(1)),train_target))

In [18]:
from d2l import torch as d2l

In [19]:
def get_k_fold_data(k, i, X, y):
    assert k > 1
    fold_size = X.shape[0] // k
    X_train, y_train = None, None
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size)
        X_part, y_part = X[idx, :], y[idx]
        if j == i:
            X_valid, y_valid = X_part, y_part
        elif X_train is None:
            X_train, y_train = X_part, y_part
        else:
            X_train = torch.cat([X_train, X_part], 0)
            y_train = torch.cat([y_train, y_part], 0)
    return X_train, y_train, X_valid, y_valid

In [20]:
epochs=2000
scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max = (epochs // 9) + 1)
k_fold=5
batch_size=64
for i in range(k_fold):
    i_data=get_k_fold_data(k_fold,i,train_sentense_wordembedding,train_target)
    data_iter = d2l.load_array((i_data[0], i_data[1]), batch_size)
    for epoch in range(epochs):
        for x,y in data_iter:
            optimizer.zero_grad()
            l=loss(net(x.unsqueeze(1)),y)
            l.backward()
            optimizer.step()
            scheduler.step()
        if (epoch+1)%50==0:
            print(loss(net(i_data[2].unsqueeze(1)),i_data[3]))

tensor(0.6674, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6649, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6573, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6517, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6450, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6374, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6295, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6228, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6162, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6124, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6047, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6017, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5960, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5919, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5872, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5832, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5791, device='cuda:0', grad_fn=

In [21]:
net.eval()
ttt=net(test_sentence_wordembedding.to(device).unsqueeze(1)).argmax(axis=1)
ttt=ttt.cpu().detach().numpy()
ttt=pd.DataFrame(ttt)
ttt.to_csv('F:/Kaggle/NLP/Natural Language Processing with Disaster Tweets/CNN-submission.csv')